<a href="https://colab.research.google.com/github/Nickguild1993/Gun_Violence_Exploration/blob/main/Gun_Violence_EDA_ScratchV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Having done the necessary concatenating, joining, and other cleaning tasks in the https://colab.research.google.com/drive/1_PWEtgswXbX_ntUsptZO14rzRjePT61Z#scrollTo=J7d9Dt78H_92 notebook, this one will focus on analyzing the cleaned DataFrame.

Pivot Tables / Cross Tab / Side Table work - > going to create a V3 focusing on breaking down the dataframes for visualizations

In [1]:
# import libraries

import pandas as pd
import numpy as np

from scipy import stats

! pip install sidetable
import sidetable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Get visualization libraries & datetime

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

from datetime import datetime as dt 
import datetime

In [3]:
# Pull the dataframe

url = "https://raw.githubusercontent.com/Nickguild1993/Gun_Violence_Exploration/main/cleaned_gun_violence_dataframe.csv"
df = pd.read_csv(url)
df = df.drop(columns=(["Unnamed: 0"]))
df.head(3)

,Year,State,Population,Deaths Per 100k,City,# Killed,# Injured,# Victims,Gun Law Rank,Safety Grade,Incident Date
0,2014,Louisiana,4644013,21.31,New Orleans,0,4,4,32.0,F,12/29/2014
1,2014,California,38596972,7.45,Los Angeles,1,3,4,1.0,A,12/27/2014
2,2014,California,38596972,7.45,Sacramento,0,4,4,1.0,A,12/27/2014


There isn't Safety Grade / Gun Law Rank / Deaths per 100k data for district of columbia :/

In [4]:
df = df[df["Gun Law Rank"].notna()]
# getting rid of the D.C. observations b/c there isn't data for them.

In [5]:
# Quick double check for any missing / messy data

print("shape of the dataframe:", df.shape)
print("---------------------------------")
print("missing values: ", df.isnull().sum())

shape of the dataframe: (3550, 11)
---------------------------------
missing values:  Year               0
State              0
Population         0
Deaths Per 100k    0
City               0
# Killed           0
# Injured          0
# Victims          0
Gun Law Rank       0
Safety Grade       0
Incident Date      0
dtype: int64


creating aggregation dictionaries for groupby usage

In [6]:
agg_dict = {
    "# Killed" : ["sum", "mean", "max"],
    "# Injured" : ["sum", "mean", "max"],
    "# Victims" : ["sum", "mean", "max"]
}

#######

agg_killed = {
    "# Killed" : ["sum", "mean", "max"]
}

agg_injured = {
    "# Injured" : ["sum", "mean", "max"]
}

agg_victims = {
    "# Victims" : ["sum", "mean", "max"]
}

In [7]:
df_group = df.groupby(["State"]).agg(agg_dict).reset_index().round(2)
df_group.columns

MultiIndex([(    'State',     ''),
            ( '# Killed',  'sum'),
            ( '# Killed', 'mean'),
            ( '# Killed',  'max'),
            ('# Injured',  'sum'),
            ('# Injured', 'mean'),
            ('# Injured',  'max'),
            ('# Victims',  'sum'),
            ('# Victims', 'mean'),
            ('# Victims',  'max')],
           )

In [8]:
df_killed = df.groupby(["State"]).agg(agg_killed).reset_index().round(2)

df_injured = df.groupby(["State"]).agg(agg_injured).reset_index().round(2)

df_victims = df.groupby(["State"]).agg(agg_victims).reset_index().round(2)

creating individual dataframes for killed/injured/victims

In [9]:
df_killed = df_killed.sort_values(by = [("# Killed", "sum")], ascending = False)

df_injured = df_injured.sort_values(by =[("# Injured", "sum")], ascending = False)

df_victims = df_victims.sort_values(by = [("# Victims", "sum")], ascending = False)

In [10]:
df_killed[:5]

State # Killed          
                    sum  mean max
40       Texas      419  1.72  27
4   California      394  1.16  16
8      Florida      285  1.31  50
11    Illinois      257  0.69   6
9      Georgia      149  1.09   8

Grouping by Safety Grade

In [11]:
df_grade = df.groupby(["Safety Grade"]).agg(agg_dict).reset_index().round(2)
df_grade = df_grade.set_index("Safety Grade")
df_grade

# Killed           # Injured            # Victims           
                  sum  mean max       sum  mean  max       sum  mean  max
Safety Grade                                                             
A                 942  0.83  16      4691  4.14   19      5633  4.97   35
B                  71  1.25   5       199  3.49    8       270  4.74    8
C                 863  1.19  59      3429  4.71  441      4292  5.90  500
D                 533  1.03  13      2067  3.99   18      2600  5.02   27
F                1394  1.25  27      4462  4.01   26      5856  5.26   47

The above output doesn't take into account the population for those groupings, need to sum the populations for each state in the grades to get rate statistics

In [12]:
df.head(1)

,Year,State,Population,Deaths Per 100k,City,# Killed,# Injured,# Victims,Gun Law Rank,Safety Grade,Incident Date
0,2014,Louisiana,4644013,21.31,New Orleans,0,4,4,32.0,F,12/29/2014


In [ ]:
df.pivot_table(index=["Safety Grade", "State"])

# Injured  # Killed  # Victims  Deaths Per 100k  \
Safety Grade State                                                             
A            California       3.970588  1.158824   5.129412             7.45   
             Connecticut      4.444444  0.518519   4.962963             4.91   
             Illinois         4.262032  0.687166   4.949198            10.78   
             Maryland         3.814815  0.888889   4.703704            11.61   
             Massachusetts    4.193548  0.580645   4.774194             3.46   
             New Jersey       4.206897  0.724138   4.931034             4.75   
             New York         4.331325  0.602410   4.933735             4.03   
B            Delaware         3.764706  0.764706   4.529412            11.55   
             Rhode Island     5.500000  0.000000   5.500000             3.28   
             Washington       3.138889  1.611111   4.750000            10.32   
C            Colorado         3.788462  1.384615   5.173077            15.14   
             Florida          4.412844  1.307339   5.720183            12.81   
             Iowa             4.388889  0.722222   5.111111             8.62   
             Michigan         3.750000  1.027778   4.777778            12.82   
             Minnesota        4.625000  0.800000   5.425000             7.79   
             Nebraska         4.545455  0.818182   5.363636             9.05   
             Nevada          19.275862  2.965517  22.241379            17.84   
             New Mexico       2.904762  1.857143   4.761905            20.75   
             Oregon           4.400000  1.200000   5.600000            11.68   
             Pennsylvania     4.029940  0.814371   4.844311            12.47   
             Vermont          0.000000  4.000000   4.000000            12.63   
             Wisconsin        3.953488  1.209302   5.162791            10.13   
D            Indiana          3.840000  1.093333   4.933333            14.71   
             North Carolina   3.676471  1.156863   4.833333            13.26   
             Ohio             4.241135  1.035461   5.276596            13.05   
             Tennessee        4.150442  0.814159   4.964602            17.82   
             Utah             2.833333  2.166667   5.000000            13.14   
             Virginia         3.950617  1.012346   4.962963            11.76   
F            Alabama          3.873684  1.031579   4.905263            21.70   
             Alaska           3.600000  1.200000   4.800000            20.74   
             Arizona          3.594595  1.756757   5.351351            15.29   
             Arkansas         4.930233  0.953488   5.883721            18.96   
             Georgia          3.868613  1.087591   4.956204            15.72   
             Idaho            3.000000  2.500000   5.500000            16.61   
             Kansas           4.000000  1.476190   5.476190            14.65   
             Kentucky         4.119048  0.904762   5.023810            16.81   
             Louisiana        4.303571  0.809524   5.113095            21.31   
             Maine            0.666667  4.000000   4.666667            10.32   
             Mississippi      3.671429  1.357143   5.028571            22.81   
             Missouri         3.760684  1.094017   4.854701            21.34   
             Montana          1.500000  3.250000   4.750000            17.27   
             New Hampshire    4.000000  0.000000   4.000000            10.66   
             Oklahoma         3.681818  0.909091   4.590909            16.71   
             South Carolina   4.161290  1.172043   5.333333            17.51   
             South Dakota     1.500000  3.500000   5.000000            13.50   
             Texas            4.172131  1.717213   5.889344            12.22   
             West Virginia    2.400000  2.600000   5.000000            18.11   
             Wyoming          2.000000  2.000000   4.000000            21.09   

                            

In [16]:
df.stb.freq(["State"])[:5]

,State,count,percent,cumulative_count,cumulative_percent
0,Illinois,374,10.535211,374,10.535211
1,California,340,9.577465,714,20.112676
2,Texas,244,6.873239,958,26.985915
3,Florida,218,6.140845,1176,33.126761
4,Louisiana,168,4.732394,1344,37.859155


In [18]:
df.head(1)

,Year,State,Population,Deaths Per 100k,City,# Killed,# Injured,# Victims,Gun Law Rank,Safety Grade,Incident Date
0,2014,Louisiana,4644013,21.31,New Orleans,0,4,4,32.0,F,12/29/2014


Grouping States by Safety Grade - looking at population / deaths | w/ years 

In [28]:
pd.crosstab(df["State"], df["# Killed"], margins = True, margins_name= "Total", values = df["# Killed"], aggfunc = "sum").round(0)[:5]

# Killed,0,1,2,3,4,5,6,7,8,9,...,11,13,16,17,22,23,27,50,59,Total
State,,,,,,,,,,,,,,,,,,,,,
Alabama,0.0,35.0,28.0,18.0,NaN,10.0,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98
Alaska,0.0,2.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
Arizona,0.0,13.0,12.0,9.0,16.0,15.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65
Arkansas,0.0,20.0,2.0,6.0,8.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41
California,0.0,104.0,88.0,54.0,48.0,30.0,24.0,7.0,NaN,NaN,...,NaN,13.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,394


In [48]:
pd.crosstab(df["Safety Grade"], df["# Killed"],
            margins = True,
            margins_name = 'Total',
            normalize = "index")

# Killed,0,1,2,3,4,5,6,7,8,9,10,11,13,16,17,22,23,27,50,59
Safety Grade,,,,,,,,,,,,,,,,,,,,
A,0.513680,0.316858,0.092674,0.032657,0.022948,0.010591,0.006178,0.000883,0.000000,0.000000,0.001765,0.000000,0.000883,0.000883,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B,0.473684,0.210526,0.122807,0.070175,0.035088,0.087719,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
C,0.471154,0.298077,0.109890,0.050824,0.028846,0.019231,0.009615,0.002747,0.001374,0.000000,0.002747,0.001374,0.000000,0.000000,0.001374,0.000000,0.000000,0.000000,0.001374,0.001374
D,0.476834,0.289575,0.100386,0.067568,0.038610,0.013514,0.003861,0.001931,0.001931,0.001931,0.001931,0.000000,0.001931,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
F,0.409336,0.298025,0.137343,0.075404,0.037702,0.022442,0.007181,0.001795,0.004488,0.002693,0.000898,0.000000,0.000000,0.000000,0.000000,0.000898,0.000898,0.000898,0.000000,0.000000
Total,0.466197,0.301408,0.111831,0.055493,0.031268,0.017746,0.006761,0.001690,0.001972,0.001127,0.001690,0.000282,0.000563,0.000282,0.000282,0.000282,0.000282,0.000282,0.000282,0.000282


In [61]:
df.stb.freq(["Safety Grade"], value = "# Killed")

,Safety Grade,# Killed,percent,cumulative_# Killed,cumulative_percent
0,F,1394,36.655272,1394,36.655272
1,A,942,24.769918,2336,61.425191
2,C,863,22.692611,3199,84.117802
3,D,533,14.015251,3732,98.133053
4,B,71,1.866947,3803,100.000000


In [54]:
year_table = pd.pivot_table(df, index = ["Year", "Safety Grade"],
               values = ["# Killed", "# Injured", "# Victims"],
               aggfunc = [np.sum],
               fill_value = 0).reset_index()
year_table[:5]

Year Safety Grade       sum                   
                     # Injured # Killed # Victims
0  2014            A       376       71       447
1  2014            B        15        6        21
2  2014            C       198       56       254
3  2014            D       154       44       198
4  2014            F       315       93       408

In [57]:
pop_table = pd.pivot_table(df, index = ["Year", "Safety Grade"],
               values = ["Population"],
               aggfunc = [np.sum]).reset_index()
pop_table[:10]

,Year,Safety Grade,sum
,,,Population
0,2014,A,2282376697
1,2014,B,22219901
2,2014,C,623432844
3,2014,D,321164314
4,2014,F,801502962
5,2015,A,1983777257
6,2015,B,9160974
7,2015,C,902334442
8,2015,D,463421260


In [58]:
pop_table.columns

MultiIndex([(        'Year',           ''),
            ('Safety Grade',           ''),
            (         'sum', 'Population')],
           )

In [59]:
pop_table[pop_table[("sum", "Population")] == 

0     2282376697
1       22219901
2      623432844
3      321164314
4      801502962
5     1983777257
6        9160974
7      902334442
8      463421260
9     1015108562
10    2822464602
11      60256010
12     942298338
13     444708880
14    1343758268
15    2361454613
16      23226909
17     848989420
18     561924099
19    1044325949
20    2237565771
21      23537086
22    1068069959
23     431476935
24     833247899
25    2900547383
26      32407100
27     742409037
28     479497922
29    1542429211
30    3485027607
31      41496249
32    1586631125
33     780795555
34    1790709392
35    3878288198
36      63633943
37    1657846178
38     962422594
39    2526248088
40    1179330320
41      15775930
42     521661067
43     241868822
44     978671122
Name: (sum, Population), dtype: int64